In [1]:
import cv2 as cv
import numpy as np
import math

In [2]:
# Import du model + reglage
model = 'yolov2.weights'
config = 'yolov2.cfg'
net = cv.dnn.readNetFromDarknet(config,model)
min_confidence = 0.25 # float betwen 0 and 1
probability_index = 5 # integer

In [3]:
cam = cv.VideoCapture(0)
cam.release()
cam = cv.VideoCapture(0)
font = cv.FONT_HERSHEY_COMPLEX

In [4]:
# Function to calculate the distance of object from the camera lense
def dist_calculator(startX, startY, endX, endY, box_width, box_height, img_w, img_h):
    x_3, y_3 = startX, startY - (box_height / 7)  # top left of the triangle
    # assumption: camera is rasied above the ground so considering 90% of the height of the image height
    x_1, y_1 = img_w / 2, 0.9 * img_h  # bottom of the triangle
    x_2, y_2 = endX, endY - (box_height / 7)  # top right of the triangle

    # find the angle between bottom and right point
    angle_x1_x2 = math.degrees(math.atan2(x_1 - x_2, y_1 - y_2))
    # find the angle between bottom and left point
    angle_x1_x3 = math.degrees(math.atan2(x_1 - x_3, y_1 - y_3))

    angle_right = 90 + angle_x1_x2
    angle_left = 90 - angle_x1_x3

    # total angle of view for the bench from bottom center point of the image.
    total_angle = angle_right + angle_left

    # Bench length assumed to be 2 meters in millimeters. This value can automated, based on the type of bench used.
    bench_length = 2000

    distance = (bench_length * (1 / total_angle) * 57) / 1000
    return total_angle, distance

In [5]:
# Camera
while True:
    # Import capture of video then flip
    isTrue, original_frame = cam.read()
    original_frame = cv.flip(original_frame, 1)
    # Resizing image
    resize_img = cv.resize(original_frame, (224, 224))
    # Saves
    height, width, _ = original_frame.shape
    # ============
    # Prepocess data
    blob = cv.dnn.blobFromImage(resize_img, 1.0 / 255.0, (224, 224), True, crop=False)
    # Entries + prédictions
    net.setInput(blob)
    predictions = net.forward()
    for i in range(predictions.shape[0]):
        prob_arr = predictions[i][probability_index:]
        class_index = prob_arr.argmax(axis=0)
        confidence = prob_arr[class_index]
        if confidence > min_confidence:
            x_center = predictions[i][0] * width
            y_center = predictions[i][1] * height
            width_box = predictions[i][2] * width
            height_box = predictions[i][3] * height
            
            x1 = int(x_center - width_box * 0.5)  # Start X coordinate
            y1 = int(y_center - height_box * 0.5)  # Start Y coordinate
            x2 = int(x_center + width_box * 0.5)  # End X coordinate
            y2 = int(y_center + height_box * 0.5)  # End y coordinate
            
            cv.rectangle(original_frame, (x1, y1), (x2, y2), (0, 255, 120), 2)
            
            roi_corners = [[0.0 * width_box, 1.0 * height],  # left, down
                            [x1 + (width_box / 5), y2 - (height_box / 7)],  # left, up
                            [x1 + 4 * (width_box / 5), y2 - (height_box / 7)],  # right, up
                            [1.0 * width, 1.0 * height]]  # right, down

            original_frame = cv.circle(original_frame, (int(width), int(height)), 3, (0, 0, 255), -1)
            triangle_pts = [[width / 2, 0.9 * height],  # left, down / bottom point of the traingle
                            [x1, y2 - (height_box / 7)],  # left, up
                            [x2, y2 - (height_box / 7)],  # right, up
                            [width / 2, 0.9 * height]]  # right, down / / bottom point of the traingle
            src = np.float32(triangle_pts)
            pts = np.array(src, np.int32)
            pts = pts.reshape((-1, 1, 2))
            
            _, distance = dist_calculator(x1, y1, x2, y2, width_box, height_box, width, height)
            cv.putText(original_frame, "Distance: {} mm".format(round(distance, 2)), (x1, y1 + 13), cv.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 120), 2)
    # ================ Show frame
    cv.imshow("Video", original_frame)
    # If q pressed then quit
    if cv.waitKey(20) & 0xFF == ord('q'):
        break
# Destroy all window
cv.destroyAllWindows()